# Azure OpenAI Service - Q&A with semantic answering exerise

In this tutorial, you'll build a simple Q&A system, that can give semantic answers to questions. Three sample documents from the Azure documentation are provided. Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

In [ ]:
pip install openai==0.28.1

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import tiktoken
import openai
import numpy as np
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity

# Load environment variables
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_ENCODING = 'cl100k_base'
EMBEDDING_CHUNK_SIZE = 8000
COMPLETION_MODEL = 'text-davinci-003'

# initialize tiktoken for encoding text
encoding = tiktoken.get_encoding(EMBEDDING_ENCODING)

Next, let's read the documents in `/data/qna/*.txt`, which are our sample documents:

In [ ]:
# list all files in the samples directory
samples_dir = os.path.join(os.getcwd(), "./data/qna/")
sample_files = os.listdir(samples_dir)

# read each file and remove and newlines (better for embeddings later)
documents = []
for file in sample_files:
    with open(os.path.join(samples_dir, file), "r") as f:
        content = f.read()
        content = content.replace("\n", " ")
        content = content.replace("  ", " ")
        documents.append(content)

# print some stats about the documents
print(f"Loaded {len(documents)} documents")
for doc in documents:
    num_tokens = len(encoding.encode(doc))
    print(f"Content: {doc[:80]}... \n---> Tokens: {num_tokens}\n")

Now that we have all documents loaded, we can embed them using our embedding model:

In [ ]:
# Create embeddings for all docs
embeddings = #FIXME

# print some stats about the embeddings
for e in embeddings:
    print(len(e))

Now that we have our embeddings, we can try to ask some questions and see if it retrieves the correct document. You can try the following questions:

* what is azure openai service?
* can translator be fine tuned?
* what is the difference between luis and clu?
* what is form recognizer? (should yield no result)

In [ ]:
# create embedding for question
question = "what is azure openai service?"
qe = #FIXME

# calculate cosine similarity between question and each document
similaries = #FIXME

# Get the matching document, in this case we just use argmax of similarities
max_i = #FIXME

# print some stats about the similarities
for i, s in enumerate(similaries):
    print(f"Similarity to {sample_files[i]} is {s}")
print(f"Matching document is {sample_files[max_i]}")

In [2]:
# Generate a prompt that we use for completion, in this case we put the matched document and the question in the prompt
prompt = #FIXME

# get response from completion model
response = #FIXME
answer = #FIXME

# print the question and answer
print(f"Question was: {question}\nRetrieved answer was: {answer}")


SyntaxError: invalid syntax (1899838685.py, line 2)

In [ ]:
from openai import AzureOpenAI
import os
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type

from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING = 'cl100k_base'

# let's make sure to not retry on an invalid request, because that is what we want to demonstrate
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.BadRequestError))
def get_embedding(text_or_tokens, model=EMBEDDING_MODEL):
    return client.embeddings.create(input=text_or_tokens, model=model).data[0].embedding

In [ ]:
import os
from openai import AzureOpenAI
 
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
      api_version="2024-02-01",
     azure_endpoint="https://ia242-m5z9tor6-swedencentral.openai.azure.com/"
)
# This will correspond to the custom name you chose for your deployment when you deployed a model.
# Use a gpt-35-turbo-instruct deployment.
deployment_name = "davinci-002"
 
# Send a completion call to generate an answer
prompt = ""
response = client.completions.create(
   model=deployment_name,
   prompt=prompt,
   temperature=1,
   max_tokens=100,
   top_p=0.5,
    frequency_penalty=0,
      presence_penalty=0,
     best_of=1,
      stop=None
  )
 
print(prompt + response.choices[0].text)

Great, that worked. Now we should have a simple understanding how Q&A can work using Azure OpenAI Service embeddings and completions. Next step would be:

* Chunking of longer documents (you might run into token limits for embeddings and the answering prompt)
* Usage of a vector database (pinecone, redis, etc.) to scale the search part to a larger amount of documents
* Evaluation of the top k results, instead of just the best matching document
* ...and a few more!